# Module 11 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Scraping
from splinter import Browser
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# requests
import requests
import time

# parsing
from bs4 import BeautifulSoup

# data science
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Open a Chrome window using Splinter
my_service = Service(executable_path=ChromeDriverManager().install())
browser = Browser('chrome', service=my_service)


### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)


### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
table = soup.find("table" , {"class": "table"})
str(table)

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df=pd.read_html(str(table))
df=df[0]
df

In [ ]:
# Confirm DataFrame was created successfully
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df.info()

In [ ]:
# Change data types for data analysis
df["terrestrial_date"] = pd.to_datetime(df["terrestrial_date"])

In [ ]:
# Confirm type changes were successful by examining data types again
df.info()

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
df["month"] =df.month.sort_values()
df.month.value_counts().sort_index()

In [ ]:
# 2. How many Martian days' worth of data are there?
mask=df["id"].count()
mask

In [ ]:
# 3. What is the average low temperature by month?
avg_low_temp_month = df.groupby("month")["min_temp"].mean()
print(avg_low_temp_month)

In [ ]:
# Plot the average temperature by month
avg_low_temp_month.plot(kind='bar', color='green')
plt.xlabel('month')
plt.ylabel('Temperature in Celsius')
plt.title('Average Low Temperature by Month')
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
# Sort the average low temperatures by month in descending order
avg_low_temp_month_sorted = avg_low_temp_month.sort_values(ascending=True)

# Create a bar graph for average low temperature by month
avg_low_temp_month_sorted.plot(kind='bar', color='Navy')
plt.xlabel('Month')
plt.ylabel('Temperature in Celsius')
plt.title('Temperature by Month')
plt.show()

In [ ]:
# 4. Average pressure by Martian month
avg_pressure_month = df.groupby("month")["pressure"].mean()
print(avg_pressure_month)

In [ ]:
# Plot the average pressure by month
avg_pressure_month_sorted = avg_pressure_month.sort_values(ascending=True)
# Plot the average temperature by month
avg_pressure_month_sorted.plot(kind='bar', color='Navy')
plt.xlabel('month')
plt.ylabel('Atmospheric Pressure')
plt.title('Average Pressure by Month')
plt.show()

In [ ]:
# Calculate the number of days from the start date
df['days_from_start'] = (df['terrestrial_date'] - df['terrestrial_date'].min()).dt.days

# Plot the daily minimum temperatures against the number of days
plt.figure(figsize=(12, 8))
plt.plot(df['days_from_start'], df['min_temp'], color='b')
plt.xlabel('Number of terrestrial days')
plt.ylabel('Minimum Temperature')
plt.title('Temperature on Mars')
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV

# Specify the output file name
output_file = 'output.csv'

# Write the DataFrame to a CSV file
df.to_csv(output_file, index=False)

In [ ]:
browser.quit()